## Environment Setup

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# downloading packages for running the notebook
import sys
import subprocess



In [3]:
!pip install sentence-transformers

In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from sentence_transformers import SentenceTransformer

m2 = SentenceTransformer('sentence-transformers/LaBSE')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### OpenAI Setup

Add openai key for gpt-3.5 model usage.

In [2]:
import openai

api_key = 'aa-m9TQcsmZMErvs4oFI0Si7zcCZCBj1oxTn92Ikperiy7oOfS1'

### Dataset Cleaning and Preprocessing Steps

In [3]:
with open('fa.txt', 'r') as f:
    data = f.read()

data = data.replace('\n\n','\n')

In [4]:
ques_ans = dict()


In [5]:
import pandas as pd

qa_dict = dict()

qa_dict['text'] = []

import numpy as np

# تنظیم تعداد خطوطی که باید با هم ترکیب شوند
lines_per_group = 1
grouped_sentences = []

# ترکیب خطوط و ایجاد گروه‌ها
temp_text = ""
for i, line in enumerate(data.split("\n")):
    temp_text += line + " "
    # هر ده خط یک بار به لیست اضافه می‌شود
    if (i + 1) % lines_per_group == 0:
        grouped_sentences.append(temp_text.strip())
        temp_text = ""

# در صورتی که خطوط باقی‌مانده‌ای وجود داشته باشد، آن‌ها را نیز اضافه کنید
if temp_text:
    grouped_sentences.append(temp_text.strip())
qa_dict['text']=grouped_sentences





In [6]:
from datasets import Dataset
qa_df = pd.DataFrame.from_dict(qa_dict)
# You can load a Dataset object this way
dataset = Dataset.from_pandas(qa_df)

dataset

Dataset({
    features: ['text'],
    num_rows: 68615
})

### Text Preprocessing Functions

In [7]:
## Embedding Generation Function

import numpy as np
def sent_check2(example):

    output = np.zeros([768,], dtype=np.float32)
    try:
        sent_embed = m2.encode(example['text'])
        output = sent_embed
    except:
        print("Error")

    example['embeddings'] = output

    return example

In [8]:

dataset = dataset.map(sent_check2)

Map:   0%|          | 0/68615 [00:00<?, ? examples/s]

In [9]:
from datasets import Dataset

# Assuming your dataset is already created with 'text' and 'embeddings' columns
dataset.save_to_disk('./')


Saving the dataset (0/1 shards):   0%|          | 0/68615 [00:00<?, ? examples/s]

In [10]:
pip install faiss-gpu

In [1]:
from datasets import load_from_disk

# Load the dataset
dataset = load_from_disk('./')


In [5]:
dataset.add_faiss_index(column='embeddings')

  0%|          | 0/69 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 68615
})

In [6]:
question = "افردادی که اموال خود را در راه خدا انفاق میکنند شبیه چه هستند؟"

question_embedding = m2.encode(question)
scores, retrieved_examples = dataset.get_nearest_examples('embeddings', question_embedding, k=20)
print(scores)
for exp in retrieved_examples['text']:  # Accessing the first result set

    print(exp)

    print("*****************************")

[0.8048256  0.8139429  0.81682247 0.81761307 0.8293012  0.83510286
 0.8451639  0.8495815  0.85269547 0.85575455 0.8636742  0.868536
 0.873457   0.87373126 0.8769057  0.8774164  0.8798264  0.88191986
 0.887477   0.887477  ]
همان کس که مال خود را (در راه خدا) می‌بخشد تا پاک شود.
*****************************
از تو مى‌پرسند كه چه انفاق كنند؟ بگو آنچه از مال خود انفاق مى‌كنيد، براى پدر و مادر و خويشاوندان و يتيمان و مسكينان و رهگذران باشد، و هر كار نيكى كه كنيد خدا به آن آگاه است.
*****************************
از تو مى‌پرسند چه چيز انفاق كنند؟ بگو: آنچه از مال انفاق مى‌كنيد براى پدر و مادر و خويشان و يتيمان و نيازمندان درمانده و در راه‌ماندگان باشد، و هر كار نيكى كه كنيد خدا به آن داناست.
*****************************
از تو سؤال کنند که چه انفاق کنند؟ بگو: هر چه از مال خود انفاق کنید درباره پدر و مادر و خویشان و یتیمان و فقیران و در راه ماندگان رواست، و آنچه نیکویی کنید خداوند بر آن آگاه است.
*****************************
آيا نديده‌اى آنهايى كه در آيات خدا مجادله مى‌كنند چگونه منحرف مى‌شون

In [9]:
question = "هنگامی که اصحاب موسی به یک نوع طعام اعتراض کردند موسی به آنها چه گفت؟"

question_embedding = m2.encode(question)
scores, retrieved_examples = dataset.get_nearest_examples('embeddings', question_embedding, k=20)

In [10]:
# Assuming hits is a list of dictionaries with 'corpus_id' as a key
for exp in retrieved_examples['text']:  # Accessing the first result set

    print(exp)

    print("*****************************")



هنگامی که ساحران (به میدان مبارزه) آمدند، موسی به آنها گفت: «آنچه (از وسایل سحر) را می‌توانید بیفکنید، بیفکنید!»
*****************************
«موسی! چه چیز تورا از (برابر) قومت (سوی طور) شتابان کرد؟»
*****************************
هنگامی که به پدرش وقومش گفت: چه چیز را می پرستید؟
*****************************
موسی به آنان گفت: «آنچه را شما افکنندگانید بیفکنید.»
*****************************
موسى گفت: اى هارون! وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد
*****************************
پس چون جادوگران آمدند، موسی به آنان گفت: «آنچه را شما اندازنده‌اید بیندازید.»
*****************************
و چون جادوگران آمدند، موسى به آنان گفت: «آنچه را مى‌اندازيد بيندازيد.»
*****************************
آنگاه كه به پدرش و قومش گفت: اين تنديسها كه به پرستش آنها دل نهاده‌ايد چيستند؟
*****************************
آن‌گاه موسی گفت: ای سامری این فتنه چه بود که تو برپا کردی؟
*****************************
اى موسى، چه چيز تو را واداشت تا بر قومت پيشى گيرى؟
*****************************
موسى به آنان گفت: آنچه ر

In [11]:
scores

array([0.73792523, 0.7889748 , 0.7910744 , 0.8060277 , 0.82429916,
       0.8273556 , 0.83299756, 0.8330201 , 0.8357699 , 0.83618456,
       0.83803177, 0.8382557 , 0.84735835, 0.85079306, 0.8530512 ,
       0.8546027 , 0.855188  , 0.85580665, 0.85757035, 0.85888124],
      dtype=float32)

In [12]:

import torch
from sentence_transformers import util

# Ensure that question_embedding is a tensor
question_embedding_tensor = torch.tensor([question_embedding])

# Convert dataset['embeddings'] to a tensor if it's a list
embeddings_tensor = torch.tensor(dataset['embeddings'])
hits = util.semantic_search(question_embedding, embeddings_tensor)
hits

<ipython-input-12-52f49fedcaed>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  question_embedding_tensor = torch.tensor([question_embedding])


[[{'corpus_id': 45095, 'score': 0.6310373544692993},
  {'corpus_id': 14902, 'score': 0.6055123805999756},
  {'corpus_id': 21709, 'score': 0.6044628620147705},
  {'corpus_id': 15446, 'score': 0.5969861149787903},
  {'corpus_id': 52327, 'score': 0.5878505706787109},
  {'corpus_id': 13915, 'score': 0.5863223075866699},
  {'corpus_id': 38859, 'score': 0.583501398563385},
  {'corpus_id': 8770, 'score': 0.5834899544715881},
  {'corpus_id': 2442, 'score': 0.5821152329444885},
  {'corpus_id': 8666, 'score': 0.5819077491760254}]]

In [13]:
# Assuming hits is a list of dictionaries with 'corpus_id' as a key
for hit in hits[0]:  # Accessing the first result set
    corpus_id = hit['corpus_id']
    print(dataset["text"][corpus_id])
    print(hit['score'])
    print("*****************************")



هنگامی که ساحران (به میدان مبارزه) آمدند، موسی به آنها گفت: «آنچه (از وسایل سحر) را می‌توانید بیفکنید، بیفکنید!»
0.6310373544692993
*****************************
«موسی! چه چیز تورا از (برابر) قومت (سوی طور) شتابان کرد؟»
0.6055123805999756
*****************************
هنگامی که به پدرش وقومش گفت: چه چیز را می پرستید؟
0.6044628620147705
*****************************
موسی به آنان گفت: «آنچه را شما افکنندگانید بیفکنید.»
0.5969861149787903
*****************************
موسى گفت: اى هارون! وقتى ديدى آنها گمراه شدند چه چيز مانع تو شد
0.5878505706787109
*****************************
پس چون جادوگران آمدند، موسی به آنان گفت: «آنچه را شما اندازنده‌اید بیندازید.»
0.5863223075866699
*****************************
و چون جادوگران آمدند، موسى به آنان گفت: «آنچه را مى‌اندازيد بيندازيد.»
0.583501398563385
*****************************
آنگاه كه به پدرش و قومش گفت: اين تنديسها كه به پرستش آنها دل نهاده‌ايد چيستند؟
0.5834899544715881
*****************************
آن‌گاه موسی گفت: ای سامری این فتنه چه بود ک

In [ ]:

updated_message = [
        {"role": "system", "content": "I want you to act as a question answering bot which uses the context mentioned and answer in a concise manner in persian and doesn't make stuff up."
                                      "You will answer question based on the context - {}"
                                     },
        {"role": "user", "content": "Now I want you to answer this question {}."},
    ]

updated_message[0]['content'] = updated_message[0]['content'].format(retrieved_examples['text'])
updated_message[1]['content'] = updated_message[1]['content'].format(question)

completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=updated_message,
                                                      timeout=240, max_tokens=400, n=1, stop=None, temperature=0)

In [ ]:
completion['choices'][0]['message']['content']